In [178]:
# def compute_overlap_score(path1, path2, beta=0.01, delta=0):
#     """
#     Calculate an overlap score between two paths, inspired by the Jaccard Index,
#     with a small penalty for unshared edges.

#     Parameters:
#         path1 (list): The first path as a sequence of nodes.
#         path2 (list): The second path as a sequence of nodes.
#         beta (float): Weight for penalizing unshared edges.
#         delta (float): Baseline adjustment to ensure non-negativity (optional).

#     Returns:
#         float: Overlap score.
#     """
#     # Get edges for both paths
#     edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
#     edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
#     # Calculate shared and unshared edges
#     shared_edges = edges1 & edges2
#     unshared_edges = edges1 ^ edges2  # Symmetric difference
#     total_edges = edges1 | edges2    # Union of all edges

#     # Compute Jaccard Index-inspired score with penalty for unshared edges
#     jaccard_score = len(shared_edges) / len(total_edges) if total_edges else 0
#     penalty = beta * len(unshared_edges) / len(total_edges) if total_edges else 0
#     score = jaccard_score - penalty

#     # Add baseline adjustment to ensure non-negativity
#     # score = max(0, score + delta)

#     return score


import random
import numpy as np
zero_count = 0
noshare_count = 0

def compute_overlap_score(path1, path2, alpha=3, beta=1):
    """
    Calculate the overlap score between two paths based on shared and unshared edges.

    Parameters:
        path (list): The first path as a sequence of nodes.
        other_path (list): The second path as a sequence of nodes.
        alpha (float): Weight for shared edges (positive contribution).
        beta (float): Weight for unshared edges (negative contribution).

    Returns:
        float: Overlap score.
    """
    if len(path1) <= 1 or len(path2) <= 1:
        return 0
    # Get edges for both paths
    edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
    edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
    # Shared and unshared edges
    shared_edges = edges1 & edges2
    if len(shared_edges) == 0:
        return 0
    unshared_edges = edges1 ^ edges2 

    # Compute score
    score = alpha * len(shared_edges) - beta * len(unshared_edges) / (len(edges1) + len(edges2))
    # bound: [-beta, alpha]
    # resacle to [0, 1]
    # score = (score + beta) / (alpha + beta)
 
    return score

def split_train_test(all_paths:dict, train_ratio:float = 0.9):
    """
    all_path = [{'start_alert': int, 'end_alert': int}, ...]

    {
        'start_alert': 24,
        'end_alert': 11,
        'start_entities': [12],
        'end_entities': [9],
        'shortest_alert_path': [24, 9, 11]
    }
    """
    # construct a dictionary to map the path to the original dictionary
    random.shuffle(all_paths)
    path_to_dict = {}
    for path in all_paths:
        # extract the 0, 2, 4, 6, ... index of the path
        extracted = path['shortest_alert_path'][::2]
        path_to_dict[tuple(extracted)] = path
    
    # prepare the alert paths
    alert_paths = list(path_to_dict.keys())
    max_length = max(len(p) for p in alert_paths)
    total = len(alert_paths)
    train_len = int(total * train_ratio)
    test_len = total - train_len


    # fill the score matrix
    score_matrix = [[-1] * len(alert_paths)  for _ in range(len(alert_paths))]
    theta = 0.1 # default train length score
    for i in range(total):
        for j in range(total):
            if i == j:
                score_matrix[i][j] = 0
                continue
            if score_matrix[j][i] != -1:
                score_matrix[i][j] = score_matrix[j][i]
            score_matrix[i][j] = compute_overlap_score(alert_paths[i], alert_paths[j])
        
    # split the train and test set
    zero_ratio_count = 0
    large_ratio_count = 0
    train_keys = []
    test_keys = []
    for i in range(total):
        p = alert_paths[i]
        if train_keys == [] or test_keys == []:
            ratio = 0.5
        elif len(train_keys) >= train_len:
            test_keys += alert_paths[i:]
            break
        elif len(test_keys) >= test_len:
            train_keys += alert_paths[i:]
            break
        else:
            
            train_score = sum(score_matrix[i][alert_paths.index(k)] for k in train_keys) / len(train_keys)
            test_score = sum(score_matrix[i][alert_paths.index(k)] for k in test_keys) / len(test_keys)
            # if train_score is higher, the point should be more likely to be in test set
            try: 
                # use softmax
                exp_train = np.exp(train_score)
                exp_test = np.exp(test_score)
                ratio = exp_train / (exp_train + exp_test)

            

            except ZeroDivisionError:
                zero_ratio_count += 1
                ratio = 0.5

            # print("len train:", len(train_keys), "len test:", len(test_keys))
            # print("Train score:", train_score, "Test score:", test_score, "Ratio:", ratio)

            # add length influence
            # ratio = ratio * (theta / ( theta + len(p) / max_length))

        # weighted random selection
        if random.random() < ratio:
            train_keys.append(p)
        else:
            test_keys.append(p)

    # ----------------- compare with random selection -----------------
    total_score = 0
    for k in train_keys:
        for j in test_keys:
            total_score += score_matrix[alert_paths.index(k)][alert_paths.index(j)]
    
    # take first k as train set
    compare_train_set = []
    compare_test_set = []
    for i in range(train_len):
        if len(compare_train_set) >= train_len:
            compare_test_set += alert_paths[i:]
            break
        elif len(compare_test_set) >= test_len:
            compare_train_set += alert_paths[i:]
            break
        
        if random.random() < 0.5:
            compare_train_set.append(alert_paths[i])
        else:
            compare_test_set.append(alert_paths[i])

    compare_score = 0
    for k in compare_train_set:
        for j in compare_test_set:
            compare_score += score_matrix[alert_paths.index(k)][alert_paths.index(j)]

    print("Total score:", total_score, "Comparison score:", compare_score)
    # -----------------------------------------------------------------
    return total_score, compare_score

    # map back to original paths
    final_train_set = [path_to_dict[p1] for p1 in train_keys]
    final_test_set = [path_to_dict[p2] for p2 in test_keys]
    return final_train_set, final_test_set

In [1]:

a = {"a": 1, "b": 2}
b = {"a": 2, "b": None}
c = {**a, **b}
c

{'a': 2, 'b': None}

In [179]:

compare_score = 0
total_score = 0
for i in range(10):
    graph_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_files/incident_55.graphml"

    # load the graph
    alert_graph = AlertGraph()
    alert_graph.load_graph_from_graphml(graphfile)

    all_paths = alert_graph.get_alert_paths(verbose=False)
    if len(all_paths) < 150:
        train_ratio = 0.285
    else:
        train_ratio = 1 - 100 / len(all_paths)
        print(train_ratio)

    tscore, cscore = split_train_test(all_paths)
    total_score += tscore
    compare_score += cscore
    break

# avg
print("Average total score:", total_score / 10)
print("Average compare score:", compare_score / 10)


Total alert paths: 728. Expected: alert_num ^ 2 = 729, Selected: 728
0.8626373626373627
Total score: 1996.978571428579 Comparison score: 1735.361904761909
Average total score: 199.6978571428579
Average compare score: 173.5361904761909


## V2

In [229]:
# def compute_overlap_score(path1, path2, beta=0.01, delta=0):
#     """
#     Calculate an overlap score between two paths, inspired by the Jaccard Index,
#     with a small penalty for unshared edges.

#     Parameters:
#         path1 (list): The first path as a sequence of nodes.
#         path2 (list): The second path as a sequence of nodes.
#         beta (float): Weight for penalizing unshared edges.
#         delta (float): Baseline adjustment to ensure non-negativity (optional).

#     Returns:
#         float: Overlap score.
#     """
#     # Get edges for both paths
#     edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
#     edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
#     # Calculate shared and unshared edges
#     shared_edges = edges1 & edges2
#     unshared_edges = edges1 ^ edges2  # Symmetric difference
#     total_edges = edges1 | edges2    # Union of all edges

#     # Compute Jaccard Index-inspired score with penalty for unshared edges
#     jaccard_score = len(shared_edges) / len(total_edges) if total_edges else 0
#     penalty = beta * len(unshared_edges) / len(total_edges) if total_edges else 0
#     score = jaccard_score - penalty

#     # Add baseline adjustment to ensure non-negativity
#     # score = max(0, score + delta)

#     return score


import random
import numpy as np
zero_count = 0
noshare_count = 0

def compute_overlap_score(path1, path2, alpha=3, beta=1):
    """
    Calculate the overlap score between two paths based on shared and unshared edges.

    Parameters:
        path (list): The first path as a sequence of nodes.
        other_path (list): The second path as a sequence of nodes.
        alpha (float): Weight for shared edges (positive contribution).
        beta (float): Weight for unshared edges (negative contribution).

    Returns:
        float: Overlap score.
    """
    if len(path1) <= 1 or len(path2) <= 1:
        return 0
    # Get edges for both paths
    edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
    edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
    # Shared and unshared edges
    shared_edges = edges1 & edges2
    if len(shared_edges) == 0:
        return 0
    unshared_edges = edges1 ^ edges2 

    # Compute score
    score = alpha * len(shared_edges) - beta * len(unshared_edges) / (len(edges1) + len(edges2))
    # bound: [-beta, alpha]
    # resacle to [0, 1]
    # score = (score + beta) / (alpha + beta)
 
    return score

def split_train_test(all_paths:dict, train_ratio:float = 0.9):
    """
    all_path = [{'start_alert': int, 'end_alert': int}, ...]

    {
        'start_alert': 24,
        'end_alert': 11,
        'start_entities': [12],
        'end_entities': [9],
        'shortest_alert_path': [24, 9, 11]
    }
    """
    # construct a dictionary to map the path to the original dictionary
    random.shuffle(all_paths)
    path_to_dict = {}
    for path in all_paths:
        # extract the 0, 2, 4, 6, ... index of the path
        extracted = path['shortest_alert_path'][::2]
        path_to_dict[tuple(extracted)] = path
    
    # prepare the alert paths
    alert_paths = list(path_to_dict.keys())
    max_length = max(len(p) for p in alert_paths)
    total = len(alert_paths)
    train_len = int(total * train_ratio)
    test_len = total - train_len

    # length weight
    lweights = []
    for p in alert_paths:
        lweights.append(len(p) / max_length)
    
    # normalize the length weight
    lweights = np.array(lweights)
    lweights = lweights / lweights.sum()
    avg = lweights.mean()

    score_matrix = [[-1000] * len(alert_paths)  for _ in range(len(alert_paths))]
    def get_score(i, j):
        if score_matrix[i][j] != -1000:
            return score_matrix[i][j]
        if i==j:
            score_matrix[i][j] = 0
            return 0
        elif score_matrix[j][i] != -1000:
            score_matrix[i][j] = score_matrix[j][i]
            return score_matrix[i][j]
        score_matrix[i][j] = compute_overlap_score(alert_paths[i], alert_paths[j])
        return score_matrix[i][j]

    # split the train and test set
    train_keys = []
    test_keys = []
    for i in range(total):
        p = alert_paths[i]
        if train_keys == [] or test_keys == []:
            ratio = lweights[i] / (lweights[i] + avg)
            print("LengthRatio:", ratio)
        elif len(train_keys) >= train_len:
            test_keys += alert_paths[i:]
            break
        elif len(test_keys) >= test_len:
            train_keys += alert_paths[i:]
            break
        else:
            train_score = sum(get_score(i, alert_paths.index(k)) for k in train_keys) / len(train_keys)
            test_score = sum(get_score(i, alert_paths.index(k)) for k in test_keys) / len(test_keys)
            # if train_score is higher, the point should be more likely to be in test set

            try: 
                # use softmax
                exp_train = np.exp(train_score)
                exp_test = np.exp(test_score)
                ratio = exp_train / (exp_train + exp_test)
            except ZeroDivisionError:
                ratio = 0.5

            # print("len train:", len(train_keys), "len test:", len(test_keys))
            # print("Train score:", train_score, "Test score:", test_score, "Ratio:", ratio)

            # add length influence
            # ratio = ratio * (theta / ( theta + len(p) / max_length))

        # weighted random selection
        if random.random() < ratio:
            train_keys.append(p)
        else:
            test_keys.append(p)

    # ----------------- compare with random selection -----------------
    total_score = 0
    for k in train_keys:
        for j in test_keys:
            total_score += get_score(alert_paths.index(k), alert_paths.index(j))
    
    # take first k as train set
    compare_train_set = []
    compare_test_set = []
    for i in range(train_len):
        if len(compare_train_set) >= train_len:
            compare_test_set += alert_paths[i:]
            break
        elif len(compare_test_set) >= test_len:
            compare_train_set += alert_paths[i:]
            break
        
        if random.random() < 0.5:
            compare_train_set.append(alert_paths[i])
        else:
            compare_test_set.append(alert_paths[i])

    compare_score = 0
    for k in compare_train_set:
        for j in compare_test_set:
            compare_score += get_score(alert_paths.index(k), alert_paths.index(j))

    print("Total score:", total_score, "Comparison score:", compare_score)
    # -----------------------------------------------------------------
    return total_score, compare_score

    # map back to original paths
    final_train_set = [path_to_dict[p1] for p1 in train_keys]
    final_test_set = [path_to_dict[p2] for p2 in test_keys]
    return final_train_set, final_test_set

In [230]:
from secgym.qagen.alert_graph import AlertGraph

compare_score = 0
total_score = 0
for i in range(10):
    graph_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_files/incident_55.graphml"

    # load the graph
    alert_graph = AlertGraph()
    alert_graph.load_graph_from_graphml(graph_path)

    all_paths = alert_graph.get_alert_paths(verbose=False)
    if len(all_paths) < 150:
        train_ratio = 0.285
    else:
        train_ratio = 1 - 100 / len(all_paths)
        print(train_ratio)

    tscore, cscore = split_train_test(all_paths)
    total_score += tscore
    compare_score += cscore
    break

# avg
print("Average total score:", total_score / 10)
print("Average compare score:", compare_score / 10)


Total alert paths: 728. Expected: alert_num ^ 2 = 729, Selected: 728
0.8626373626373627
LengthRatio: 0.6451612903225806
LengthRatio: 0.5217391304347826
LengthRatio: 0.5925925925925926
LengthRatio: 0.5217391304347826
LengthRatio: 0.4210526315789473
LengthRatio: 0.5925925925925926
LengthRatio: 0.6451612903225806
LengthRatio: 0.4210526315789473
LengthRatio: 0.5217391304347826
LengthRatio: 0.5217391304347826
Total score: 2115.2976190476247 Comparison score: 2147.304761904765
Average total score: 211.52976190476247
Average compare score: 214.7304761904765


## v2.5

{23.34: 4}

In [273]:
from secgym.qagen.alert_graph import AlertGraph

compare_score = 0
total_score = 0
for i in range(10):
    graph_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_files/incident_55.graphml"

    # load the graph
    alert_graph = AlertGraph()
    alert_graph.load_graph_from_graphml(graph_path)

    all_paths = alert_graph.get_alert_paths(verbose=False)
    if len(all_paths) < 150:
        train_ratio = 0.285
    else:
        train_ratio = 1 - 100 / len(all_paths)
        print(train_ratio)

    split_train_test(all_paths)
    # total_score += tscore
    # compare_score += cscore
    break

# avg
# print("Average total score:", total_score / 10)
# print("Average compare score:", compare_score / 10)


Total alert paths: 728. Expected: alert_num ^ 2 = 729, Selected: 728
0.8626373626373627
0.382,0.392,0.299,0.3,0.303,0.379,0.393,0.276,0.316,0.343,0.187,0.279,0.214,0.317,0.33,0.34,0.2,0.29,0.304,0.342,0.38,0.29,0.312,0.278,0.351,0.343,0.283,0.285,0.267,0.276,0.246,0.376,0.234,0.418,0.286,0.326,0.404,0.316,0.292,0.315,0.348,0.242,0.325,0.365,0.249,0.275,0.322,0.336,0.243,0.264,0.328,0.291,0.325,0.242,0.285,0.303,0.282,0.361,0.279,0.341,0.3,0.27,0.299,0.231,0.419,0.252,0.499,0.253,0.334,0.232,0.369,0.436,0.212,0.231,0.265,0.366,0.348,0.386,0.35,0.275,0.391,0.32,0.288,0.323,0.331,0.372,0.307,0.345,0.402,0.283,0.48,0.288,0.398,0.22,0.218,0.264,0.238,0.342,0.291,0.387,
Best score: 2603.645238095249
[1950.8428571428622, 2401.3500000000095, 1830.3928571428596, 1679.823809523813, 1931.0285714285783, 1955.1880952380986, 2155.6452380952483, 1538.0071428571453, 2019.2904761904824, 2117.9142857142906, 1350.8738095238086, 1708.7857142857156, 1113.9833333333336, 1728.2309523809563, 1970.557142857149

In [275]:
from secgym.qagen.alert_graph import AlertGraph

compare_score = 0
total_score = 0
for i in range(10):
    graph_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_files/incident_55.graphml"

    # load the graph
    alert_graph = AlertGraph()
    alert_graph.load_graph_from_graphml(graph_path)

    all_paths = alert_graph.get_alert_paths(verbose=False)
    if len(all_paths) < 150:
        train_ratio = 0.285
    else:
        train_ratio = 1 - 100 / len(all_paths)
        print(train_ratio)

    split_train_test(all_paths)
    # total_score += tscore
    # compare_score += cscore
    break

# avg
# print("Average total score:", total_score / 10)
# print("Average compare score:", compare_score / 10)


Total alert paths: 728. Expected: alert_num ^ 2 = 729, Selected: 728
0.8626373626373627
0.356,0.454,0.304,0.366,0.332,0.245,0.424,0.426,0.324,0.297,0.283,0.291,0.237,0.368,0.265,0.299,0.283,0.315,0.499,0.419,0.319,0.341,0.345,0.442,0.234,0.329,0.314,0.289,0.264,0.276,0.313,0.303,0.353,0.381,0.313,0.361,0.333,0.257,0.332,0.266,0.263,0.428,0.241,0.288,0.253,0.368,0.378,0.226,0.4,0.28,0.265,0.304,0.262,0.291,0.336,0.3,0.243,0.289,0.303,0.361,0.282,0.434,0.324,0.381,0.359,0.281,0.352,0.411,0.312,0.321,0.39,0.335,0.382,0.303,0.338,0.263,0.279,0.429,0.254,0.234,0.246,0.276,0.354,0.391,0.341,0.247,0.339,0.373,0.352,0.356,0.297,0.388,0.244,0.306,0.367,0.337,0.332,0.285,0.236,0.376,
Best score: 2994.3166666666757
[2465.9976190476254, 2839.8238095238207, 2296.580952380953, 2340.7095238095267, 2323.9071428571497, 1931.3142857142902, 2953.145238095249, 2900.235714285721, 2454.0023809523814, 2030.3500000000004, 2042.2619047619064, 2377.5619047619125, 1693.788095238095, 2389.992857142865, 1811.47857

In [276]:
used = [2465.9976190476254, 2839.8238095238207, 2296.580952380953, 2340.7095238095267, 2323.9071428571497, 1931.3142857142902, 2953.145238095249, 2900.235714285721, 2454.0023809523814, 2030.3500000000004, 2042.2619047619064, 2377.5619047619125, 1693.788095238095, 2389.992857142865, 1811.4785714285736, 2311.0880952381, 2367.1428571428614, 2371.033333333335, 2994.3166666666757, 2912.926190476202, 2454.8261904761916, 2540.7238095238117, 2657.302380952385, 2765.659523809535, 1816.3952380952355, 2336.266666666671, 1814.3190476190503, 1982.0595238095264, 1842.6166666666688, 2107.961904761902, 2200.469047619046, 2340.2190476190503, 2566.8238095238157, 2748.000000000007, 2316.650000000003, 2655.7095238095294, 2398.0142857142896, 1754.883333333333, 2467.326190476194, 2249.604761904764, 2048.297619047619, 2710.0738095238157, 1784.052380952382, 2107.833333333332, 1766.6166666666684, 2648.1238095238205, 2790.959523809536, 1499.5333333333303, 2835.707142857153, 2185.0357142857156, 2122.0404761904765, 2327.328571428571, 1877.9880952380945, 2117.083333333334, 2486.4071428571447, 2267.3952380952337, 1865.2095238095217, 1945.95476190476, 2065.469047619043, 2326.9976190476186, 2272.259523809528, 2818.1690476190615, 2327.6047619047654, 2577.6238095238164, 2490.480952380956, 1856.7047619047628, 2530.4500000000053, 2603.7619047619123, 2176.9166666666674, 2199.202380952383, 2662.8476190476263, 2405.516666666673, 2363.4904761904786, 2051.4690476190476, 2257.1261904761923, 1985.5619047619048, 1918.5833333333321, 2637.9000000000115, 2019.1666666666667, 1805.2976190476168, 1834.1309523809537, 1744.742857142857, 2312.2357142857186, 2622.269047619057, 2387.4428571428584, 2049.580952380951, 2193.9238095238134, 2618.1500000000083, 2433.6809523809566, 2632.5261904761996, 2320.378571428576, 2940.2428571428677, 1807.2999999999984, 2229.14047619048, 2640.9523809523866, 2485.750000000001, 2356.5833333333358, 2323.7952380952383, 1806.569047619045, 2674.3738095238177]
not_used = [1950.8428571428622, 2401.3500000000095, 1830.3928571428596, 1679.823809523813, 1931.0285714285783, 1955.1880952380986, 2155.6452380952483, 1538.0071428571453, 2019.2904761904824, 2117.9142857142906, 1350.8738095238086, 1708.7857142857156, 1113.9833333333336, 1728.2309523809563, 1970.5571428571495, 2057.554761904766, 1219.238095238096, 1904.9238095238131, 1637.3714285714318, 1861.3809523809566, 2284.6547619047747, 1639.6071428571445, 1798.685714285719, 2035.1500000000071, 2049.7380952381013, 1967.573809523815, 1768.9642857142892, 1798.9666666666726, 1705.8238095238123, 1198.4476190476182, 1386.223809523812, 2048.1785714285775, 1450.5833333333337, 2330.535714285723, 1743.8428571428624, 1799.1904761904814, 2268.1357142857255, 1539.2380952380963, 1606.7952380952397, 1858.9690476190515, 2196.3809523809605, 1195.357142857142, 1770.5404761904806, 2283.833333333343, 1420.250000000003, 1679.3976190476208, 2138.6571428571515, 2070.8666666666722, 1138.0761904761903, 1768.150000000003, 1941.1476190476233, 1716.5785714285742, 2026.4761904761967, 1386.5190476190498, 1665.673809523814, 1521.0785714285737, 1690.178571428575, 2045.083333333343, 1514.195238095238, 1861.8857142857196, 1925.3928571428637, 1706.49047619048, 1453.9976190476202, 1446.4619047619087, 2037.1714285714363, 1375.6809523809545, 2462.7238095238217, 1674.7952380952413, 2143.4523809523907, 1401.0761904761926, 2123.2285714285786, 2330.933333333339, 1302.7095238095242, 1249.7809523809515, 1588.566666666668, 2132.9119047619133, 2267.0476190476274, 2176.6214285714354, 2095.642857142863, 1509.8190476190518, 2200.5238095238137, 1805.3976190476267, 1515.228571428573, 1615.6214285714302, 1966.1404761904846, 2256.6238095238173, 1856.91190476191, 2018.3380952381, 2237.259523809532, 1595.4357142857157, 2603.645238095249, 1671.914285714289, 2071.464285714289, 1117.133333333333, 1393.414285714286, 1730.3452380952447, 1567.8500000000008, 2069.900000000005, 1699.5142857142885, 2180.6523809523896]
print("Used:", sum(used) / len(used))
print("Not Used:", sum(not_used) / len(not_used))

Used: 2299.7549761904797
Not Used: 1810.198333333338


In [271]:
used = [1427.7666666666678, 1549.5857142857142, 1738.5166666666669, 1831.488095238098, 1731.5928571428597, 2157.678571428574, 2128.9357142857184, 2192.864285714291, 1574.5071428571418, 1727.204761904761, 2247.41666666667, 1517.788095238097, 2282.1452380952414, 2244.119047619051, 2173.678571428576, 2023.3309523809553, 1754.26666666667, 2253.200000000006, 1742.6809523809573, 2183.11428571429, 2234.5166666666723, 1787.8166666666673, 1947.914285714286, 1456.6880952380934, 2362.688095238105, 1694.0642857142877, 2154.309523809528, 2095.9880952381004, 1587.123809523813, 1644.7952380952406, 2499.538095238106, 2190.840476190481, 1901.9976190476225, 2282.1214285714336, 2489.1928571428657, 1606.6285714285686, 2454.780952380964, 1573.4928571428588, 2120.411904761908, 1339.5190476190505, 1907.9452380952407, 2364.4880952381013, 1808.4166666666683, 1922.7880952380951, 1915.3, 2348.3357142857185, 2047.39523809524, 1733.05238095238, 2104.7261904761926, 2557.221428571439]

b = [1898.5809523809592, 2167.719047619056, 2234.957142857154, 1898.3714285714348, 2175.1952380952443, 2004.1047619047695, 1864.685714285718, 2013.100000000006, 1550.080952380953, 1908.8238095238191, 1840.2904761904845, 2130.238095238103, 1934.1190476190545, 1468.8000000000009, 2055.871428571436, 1978.7809523809592, 1607.1000000000035, 1980.6619047619104, 1471.466666666667, 1967.2142857142937, 1869.347619047623, 1548.5380952380988, 1949.5857142857226, 1738.3523809523888, 1688.7190476190526, 1587.5142857142894, 1871.7714285714337, 2030.3904761904835, 2054.3095238095325, 1964.1142857142922, 1804.023809523816, 1354.9809523809563, 1652.0380952380974, 1807.952380952386, 1810.2476190476232, 1762.1857142857184, 1897.50952380953, 1974.2714285714349, 1716.2000000000037, 2056.666666666678, 1622.9047619047665, 2340.2380952381036, 1586.1523809523853, 2275.185714285727, 1277.2142857142846, 1579.6047619047665, 1688.4428571428634, 1850.7333333333395, 1751.6285714285757, 1677.5047619047655]

print(np.mean(used), np.std(used))
print(np.mean(b), np.std(b))

1972.3195714285746 314.6977080375041
1838.7698095238154 232.44030377041082


In [6]:
from secgym.qagen.alert_graph import AlertGraph
import os 
import json

# def compute_overlap_score(path1, path2, beta=0.01, delta=0):
#     """
#     Calculate an overlap score between two paths, inspired by the Jaccard Index,
#     with a small penalty for unshared edges.

#     Parameters:
#         path1 (list): The first path as a sequence of nodes.
#         path2 (list): The second path as a sequence of nodes.
#         beta (float): Weight for penalizing unshared edges.
#         delta (float): Baseline adjustment to ensure non-negativity (optional).

#     Returns:
#         float: Overlap score.
#     """
#     # Get edges for both paths
#     edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
#     edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
#     # Calculate shared and unshared edges
#     shared_edges = edges1 & edges2
#     unshared_edges = edges1 ^ edges2  # Symmetric difference
#     total_edges = edges1 | edges2    # Union of all edges

#     # Compute Jaccard Index-inspired score with penalty for unshared edges
#     jaccard_score = len(shared_edges) / len(total_edges) if total_edges else 0
#     penalty = beta * len(unshared_edges) / len(total_edges) if total_edges else 0
#     score = jaccard_score - penalty

#     # Add baseline adjustment to ensure non-negativity
#     # score = max(0, score + delta)

#     return score


import random
import numpy as np
import time
import json

def compute_overlap_score(path1, path2, alpha=3, beta=1):
    """
    Calculate the overlap score between two paths based on shared and unshared edges.

    Parameters:
        path (list): The first path as a sequence of nodes.
        other_path (list): The second path as a sequence of nodes.
        alpha (float): Weight for shared edges (positive contribution).
        beta (float): Weight for unshared edges (negative contribution).

    Returns:
        float: Overlap score.
    """
    if len(path1) <= 1 or len(path2) <= 1:
        return 0
    # Get edges for both paths
    edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
    edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
    # Shared and unshared edges
    shared_edges = edges1 & edges2
    if len(shared_edges) == 0:
        return 0
    unshared_edges = edges1 ^ edges2 

    # Compute score
    score = alpha * len(shared_edges) - beta * len(unshared_edges) / (len(edges1) + len(edges2))
    # bound: [-beta, alpha]
    # resacle to [0, 1]
    # score = (score + beta) / (alpha + beta)
 
    return score

def split_train_test(all_paths:dict, train_ratio:float = 0.9, trials=5):
    """
    all_path = [{'start_alert': int, 'end_alert': int}, ...]

    {
        'start_alert': 24,
        'end_alert': 11,
        'start_entities': [12],
        'end_entities': [9],
        'shortest_alert_path': [24, 9, 11]
    }
    """
    # construct a dictionary to map the path to the original dictionary
    random.shuffle(all_paths)
    path_to_dict = {}
    for path in all_paths:
        # extract the 0, 2, 4, 6, ... index of the path
        extracted = path['shortest_alert_path'][::2]
        path_to_dict[tuple(extracted)] = path
    
    # prepare the alert paths
    alert_paths = list(path_to_dict.keys())
    max_length = max(len(p) for p in alert_paths)
    total = len(alert_paths)
    train_len = int(total * train_ratio)
    test_len = total - train_len

    # length weight
    lweights = []
    for p in alert_paths:
        lweights.append(len(p) / max_length)
    lweights = np.array(lweights)
    lweights = lweights / lweights.sum()
    avg = lweights.mean()

    score_matrix = [[-1000] * len(alert_paths)  for _ in range(len(alert_paths))]

    # helper function to get the score
    def get_score(i, j):
        if score_matrix[i][j] != -1000:
            return score_matrix[i][j]
        if i==j:
            score_matrix[i][j] = 0
            return 0
        elif score_matrix[j][i] != -1000:
            score_matrix[i][j] = score_matrix[j][i]
            return score_matrix[i][j]
        score_matrix[i][j] = compute_overlap_score(alert_paths[i], alert_paths[j])
        return score_matrix[i][j]

    # random split and compare
    train_keys = []
    test_keys = []
    score_splits = {}
    for _ in range(trials):
        for i in range(total):
            if len(train_keys) >= train_len:
                test_keys += alert_paths[i:]
                break
            elif len(test_keys) >= test_len:
                train_keys += alert_paths[i:]
                break
            if random.random() < (lweights[i] / (lweights[i] + avg)):
                train_keys.append(alert_paths[i])
            else:
                test_keys.append(alert_paths[i])

        compare_score = 0
        for k in train_keys:
            for j in test_keys:
                compare_score += get_score(alert_paths.index(k), alert_paths.index(j))

        score_splits[compare_score] = (train_keys, test_keys)
        train_keys = []
        test_keys = []

    # assert len(final_train_set) + len(final_test_set) == total, f"Length mismatch: {len(final_train_set)} + {len(final_test_set)} != {total}"
    return score_splits, path_to_dict

graph_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_files"
qa_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_path"

train_total_count = 0
test_total_count = 0

median_score_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/media_split"
high_score_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/high_split"
low_score_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/low_split"

# create
os.makedirs(median_score_path, exist_ok=True)
os.makedirs(high_score_path, exist_ok=True)
os.makedirs(low_score_path, exist_ok=True)

def save_to_split(path, filename, train_keys, test_keys, path_to_dict):
    with open(os.path.join(path, filename.split(".")[0] + ".json"), "w") as f:
        train_set = [path_to_dict[p1] for p1 in train_keys]
        test_set = [path_to_dict[p2] for p2 in test_keys]
        json.dump({"train": train_set, "test": test_set}, f)

for filename in os.listdir(graph_path):
    if filename.endswith(".graphml"):
        if "_5." not in filename:
            continue
        print(filename)

        graphfile = graph_path + "/" + filename
        alert_graph = AlertGraph()
        alert_graph.load_graph_from_graphml(graphfile)
        all_paths = alert_graph.get_alert_paths(verbose=False)

        if len(all_paths) < 150:
            train_ratio = 0.288
            # trials = 20
        else:
            train_ratio = 1 - 100 / len(all_paths)
        print("Path length:", len(all_paths), "Train ratio:", train_ratio)

        score_splits, path_to_dict = split_train_test(all_paths, train_ratio, trials=50)

        # save high, low, median score
        scores = list(score_splits.keys())
        scores.sort()
        median = scores[len(scores) // 2]
        high = scores[-1]
        low = scores[0]

        save_to_split(median_score_path, filename, *score_splits[median], path_to_dict)
        save_to_split(high_score_path, filename, *score_splits[high], path_to_dict)
        save_to_split(low_score_path, filename, *score_splits[low], path_to_dict)
        
        print("Median score:", median, "High score:", high, "Low score:", low)
    

        # qafile = qa_path + "/" + filename.split(".")[0] + ".json"
        # if os.path.exists(qafile):
        #     # get best score
        #     with open(qafile, "r") as f:
        #         data = json.load(f)
        #         best_score = data["score"]
        #         print("New best score:", score, "Old best score:", best_score)
        #         if score > best_score:
        #             print("New best score is better, update")
        #             with open(qafile, "w") as f:
        #                 json.dump({"train": train_set, "test": test_set, "score": score}, f)
        #         else:
        #             print("Old best score is better, skip")
        # else:
        #     with open(qafile, "w") as f:
        #         json.dump({"train": train_set, "test": test_set, "score": score}, f)
            
        # print("Train set:", len(train_set), "Test set:", len(test_set))
        # train_total_count += len(train_set)
        # test_total_count += len(test_set)
        # print("-"*100)

print("Total test set:", test_total_count)

incident_5.graphml
Total alert paths: 4621. Expected: alert_num ^ 2 = 4624, Selected: 4621
Path length: 4621 Train ratio: 0.9783596624107336
Median score: 4536.233333333362 High score: 5395.499999999955 Low score: 2853.0333333333633
Total test set: 0


In [ ]:
question, answer -> Training

question -> Baseline -> Trajectory, Reward=1 

get all the correct trajectories -> S -> A A

9

## V3

In [ ]:
# def compute_overlap_score(path1, path2, beta=0.01, delta=0):
#     """
#     Calculate an overlap score between two paths, inspired by the Jaccard Index,
#     with a small penalty for unshared edges.

#     Parameters:
#         path1 (list): The first path as a sequence of nodes.
#         path2 (list): The second path as a sequence of nodes.
#         beta (float): Weight for penalizing unshared edges.
#         delta (float): Baseline adjustment to ensure non-negativity (optional).

#     Returns:
#         float: Overlap score.
#     """
#     # Get edges for both paths
#     edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
#     edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
#     # Calculate shared and unshared edges
#     shared_edges = edges1 & edges2
#     unshared_edges = edges1 ^ edges2  # Symmetric difference
#     total_edges = edges1 | edges2    # Union of all edges

#     # Compute Jaccard Index-inspired score with penalty for unshared edges
#     jaccard_score = len(shared_edges) / len(total_edges) if total_edges else 0
#     penalty = beta * len(unshared_edges) / len(total_edges) if total_edges else 0
#     score = jaccard_score - penalty

#     # Add baseline adjustment to ensure non-negativity
#     # score = max(0, score + delta)

#     return score


import random
import numpy as np
zero_count = 0
noshare_count = 0

def compute_overlap_score(path1, path2, alpha=3, beta=1):
    """
    Calculate the overlap score between two paths based on shared and unshared edges.

    Parameters:
        path (list): The first path as a sequence of nodes.
        other_path (list): The second path as a sequence of nodes.
        alpha (float): Weight for shared edges (positive contribution).
        beta (float): Weight for unshared edges (negative contribution).

    Returns:
        float: Overlap score.
    """
    if len(path1) <= 1 or len(path2) <= 1:
        return 0
    # Get edges for both paths
    edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
    edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
    # Shared and unshared edges
    shared_edges = edges1 & edges2
    if len(shared_edges) == 0:
        return 0
    unshared_edges = edges1 ^ edges2 

    # Compute score
    score = alpha * len(shared_edges) - beta * len(unshared_edges) / (len(edges1) + len(edges2))
    # bound: [-beta, alpha]
    # resacle to [0, 1]
    # score = (score + beta) / (alpha + beta)
 
    return score

def split_train_test(all_paths:dict, train_ratio:float = 0.9):
    """
    all_path = [{'start_alert': int, 'end_alert': int}, ...]

    {
        'start_alert': 24,
        'end_alert': 11,
        'start_entities': [12],
        'end_entities': [9],
        'shortest_alert_path': [24, 9, 11]
    }
    """
    # construct a dictionary to map the path to the original dictionary
    random.shuffle(all_paths)
    path_to_dict = {}
    for path in all_paths:
        # extract the 0, 2, 4, 6, ... index of the path
        extracted = path['shortest_alert_path'][::2]
        path_to_dict[tuple(extracted)] = path
    
    # prepare the alert paths
    alert_paths = list(path_to_dict.keys())
    max_length = max(len(p) for p in alert_paths)
    total = len(alert_paths)
    train_len = int(total * train_ratio)
    test_len = total - train_len

    # length weight
    lweights = []
    for p in alert_paths:
        lweights.append(len(p) / max_length)
    lweights = np.array(lweights)
    lweights = lweights / lweights.sum()

    score_matrix = [[-1000] * len(alert_paths)  for _ in range(len(alert_paths))]
    def get_score(i, j):
        if score_matrix[i][j] != -1000:
            return score_matrix[i][j]
        if i==j:
            score_matrix[i][j] = 0
            return 0
        elif score_matrix[j][i] != -1000:
            score_matrix[i][j] = score_matrix[j][i]
            return score_matrix[i][j]
        score_matrix[i][j] = compute_overlap_score(alert_paths[i], alert_paths[j])
        return score_matrix[i][j]
    
    # non_sampled
    remaining = alert_paths.copy()
    train_keys = []
    test_keys = []
    unassigned_keys = alert_paths.copy()
    for _ in range(total):
        if len(train_keys) >= train_len:
            break
        if len(test_keys) >= test_len:
            break
        if train_keys == [] or test_keys == []:
          sample_index = np.random.choice(1, (total), p=lweights)
        else:
            # for each unassigned, update the score afte the new sample is assigned
        # p = remaining[sample_index]




    # split the train and test set
    train_keys = []
    test_keys = []
    for i in range(total):
        p = alert_paths[i]
        if train_keys == [] or test_keys == []:
            ratio = 0.5
        elif len(train_keys) >= train_len:
            test_keys += alert_paths[i:]
            break
        elif len(test_keys) >= test_len:
            train_keys += alert_paths[i:]
            break
        else:
            train_score = sum(get_score(i, alert_paths.index(k)) for k in train_keys) / len(train_keys)
            test_score = sum(get_score(i, alert_paths.index(k)) for k in test_keys) / len(test_keys)
            # if train_score is higher, the point should be more likely to be in test set

            try: 
                # use softmax
                exp_train = np.exp(train_score)
                exp_test = np.exp(test_score)
                ratio = exp_train / (exp_train + exp_test)
            except ZeroDivisionError:
                ratio = 0.5

            # print("len train:", len(train_keys), "len test:", len(test_keys))
            # print("Train score:", train_score, "Test score:", test_score, "Ratio:", ratio)

            # add length influence
            # ratio = ratio * (theta / ( theta + len(p) / max_length))

        # weighted random selection
        if random.random() < ratio:
            train_keys.append(p)
        else:
            test_keys.append(p)

    # ----------------- compare with random selection -----------------
    total_score = 0
    for k in train_keys:
        for j in test_keys:
            total_score += get_score(alert_paths.index(k), alert_paths.index(j))
    
    # take first k as train set
    compare_train_set = []
    compare_test_set = []
    for i in range(train_len):
        if len(compare_train_set) >= train_len:
            compare_test_set += alert_paths[i:]
            break
        elif len(compare_test_set) >= test_len:
            compare_train_set += alert_paths[i:]
            break
        
        if random.random() < 0.5:
            compare_train_set.append(alert_paths[i])
        else:
            compare_test_set.append(alert_paths[i])

    compare_score = 0
    for k in compare_train_set:
        for j in compare_test_set:
            compare_score += get_score(alert_paths.index(k), alert_paths.index(j))

    print("Total score:", total_score, "Comparison score:", compare_score)
    # -----------------------------------------------------------------
    return total_score, compare_score

    # map back to original paths
    final_train_set = [path_to_dict[p1] for p1 in train_keys]
    final_test_set = [path_to_dict[p2] for p2 in test_keys]
    return final_train_set, final_test_set

In [186]:
from secgym.qagen.alert_graph import AlertGraph
import os 

graph_path = "/Users/kevin/Downloads/SecRL/secgym/qagen/graph_files"

train_total_count = 0
test_total_count = 0

for filename in os.listdir(graph_path):
    if filename.endswith(".graphml"):
        if "55" not in filename:
            continue
        graphfile = graph_path + "/" + filename
        alert_graph = AlertGraph()
        alert_graph.load_graph_from_graphml(graphfile)
        all_paths = alert_graph.get_alert_paths(verbose=False)

        if len(all_paths) < 150:
            train_ratio = 0.285
        else:
            train_ratio = 1 - 100 / len(all_paths)
            print(train_ratio)

        train_set, test_set = split_train_test(all_paths)
        print("Train set:", len(train_set), "Test set:", len(test_set))
        train_total_count += len(train_set)
        test_total_count += len(test_set)
        break


Total alert paths: 728. Expected: alert_num ^ 2 = 729, Selected: 728
0.8626373626373627
Total score: 1971.8095238095293 Comparison score: 1950.692857142863


TypeError: object of type 'float' has no len()

In [119]:
                # min_score = min(train_score, test_score)
                # if min_score < 0:
                #     train_score += abs(min_score)
                #     test_score += abs(min_score)
                # ratio = train_score / (train_score + test_score)
                # print("Ratio:", train_score, test_score, ratio)
                # if ratio > 0.55 or ratio < 0.45:
                #     large_ratio_count += 1

            # train_score = 0
            # for k in train_keys:
            #     if score_matrix[i][alert_paths.index(k)] == -1:
            #         score_matrix[i][alert_paths.index(k)] = compute_overlap_score(p, k)
            #     train_score += score_matrix[i][alert_paths.index(k)]
            
            # test_score = 0
            # for k in test_keys:
            #     if score_matrix[i][alert_paths.index(k)] == -1:
            #         score_matrix[i][alert_paths.index(k)] = compute_overlap_score(p, k)
            #     test_score += score_matrix[i][alert_paths.index(k)]

In [120]:
test_total_count

73

In [121]:
pp = split_train_test(all_path)

len train: 1 len test: 1
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 1 len test: 2
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 1 len test: 3
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 1 len test: 4
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 4
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 5
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 6
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 7
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 8
Train score: 0.0 Test score: 0.3333333333333333 Ratio: 0.4174297935376853
len train: 2 len test: 9
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 10
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 11
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 12
Train score: 0.0 Test score: 0.0 Ratio: 0.5
len train: 2 len test: 13
Train score: 0.0 Test score: 0.0 Ratio: 0.5


In [122]:
    # print(len(train_keys), len(test_keys))
    # print(train_len, total)    
    # print(len(compare_train_set), len(compare_test_set))

    # total_lenth_test = 0
    # for k in test_keys:
    #     total_lenth_test += len(k)
    # print("Total length test:", total_lenth_test)
    # total_score = 0
    
    # total_lenth_test = 0
    # for k in compare_test_set:
    #     total_lenth_test += len(k)
    # print("Total length test compare:", total_lenth_test)

In [123]:

def compute_overlap_score(path1, path2, alpha=3, beta=1):
    """
    Calculate the overlap score between two paths based on shared and unshared edges.

    Parameters:
        path (list): The first path as a sequence of nodes.
        other_path (list): The second path as a sequence of nodes.
        alpha (float): Weight for shared edges (positive contribution).
        beta (float): Weight for unshared edges (negative contribution).

    Returns:
        float: Overlap score.
    """
    global zero_count
    global noshare_count
    if len(path1) <= 1 or len(path2) <= 1:
        zero_count += 1
        return 0
    # Get edges for both paths
    edges1 = set((path1[i], path1[i + 1]) for i in range(len(path1) - 1))
    edges2 = set((path2[i], path2[i + 1]) for i in range(len(path2) - 1))
    
    # Calculate shared and unshared edges
    shared_edges = edges1 & edges2
    if len(shared_edges) == 0:
        noshare_count += 1
    unshared_edges = edges1 ^ edges2  # Symmetric difference: edges in one but not both

    # Shared and unshared edges
    shared_edges = edges1 & edges2
    unshared_edges = edges1 ^ edges2 

    # Compute score
    score = (alpha * 2 * len(shared_edges) - beta * len(unshared_edges)) / (len(edges1) + len(edges2))
    # bound: [-beta, alpha]
    # resacle to [0, 1]
    score = (score + beta) / (alpha + beta)

    return score

print(compute_overlap_score([1,2,3,4,5], [4,5]))
print(compute_overlap_score([1,2,3,4,5], [6,4,5,7]))

0.4
0.2857142857142857


In [124]:
.0142857142857142 / (.0142857142857142+.06)

0.19230769230769137

In [125]:
.0142857142857142 / (.0142857142857142+.06)

0.19230769230769137

In [126]:
len(all_path)

728

In [127]:
zero_count
# noshare_count|

0

In [128]:
zcount = 0
for i in range(len(pp)):
    for j in range(len(pp)):

        if pp[i][j] < 0:
            zcount += 1
zcount        

TypeError: '<' not supported between instances of 'dict' and 'int'

In [31]:
728*728

529984

In [16]:
for i in range(500, 700):
    print(pp[i][100:200])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [145]:
# for k, v in pp.items():
#     print(k, v)



In [142]:
zero_count

18577

In [125]:
noshare_count

242275

In [64]:
len(pp)

728

In [56]:
len(all_path)

728